# Memory / Chat

Wir haben schon das Chat-Modell von OpenAI genutzt. Allerdings haben wir das noch nicht wirklich als Chat benutzt, da die bisherigen Aufgaben stets nur ein Aufgaben + Antwortpaar produziert haben.
Wichtig zu wissen ist, dass die ganzen Sprachmodelle, die wir für Chat-Bots nutzen, keine Daten sich behalten. D.h. der Chatverlauf existiert dort nicht. Wenn man also eine Frage stellt, die sich auf vorherige Daten aus dem Verlauf beziehen, dann weiß die KI nichts mehr darüber.
Damit das Ganze trotzdem möglich ist, muss man stets die gesamte (oder Teile) der Konversation bei jedem neuen Prompt mitschicken. Der Chatverlauf wird dadurch zum Teil des Kontext und die KI hat ein Kurzzeitgedächtnis bekommen.

Erstmal ein Beispiel, wie man das zu Fuß macht. Danach schauen wir uns an, wie man das mit LangChain automatisieren kann.

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import SystemMessage, HumanMessage, AIMessage

messages = [
    SystemMessage(content="Du bist ein hilfreicher KI Assistent, der mir bei meinen Fragen und Aufgaben hilft"),
    ]

llm = ChatOpenAI()

user_prompt = "Erstelle bitte eine Liste der fünf größten Sprachmodelle" # Hier könnten wir auch tatsächlich eine Eingabeaufforderung starten. Zum Zeigen gebe ich aber den Text vor
messages.append(HumanMessage(content=user_prompt))

response = llm(messages)
messages.append(response)
print(response)

messages.append(HumanMessage(content="Welche stammen von OpenAI?"))

response = llm(messages)
messages.append(response)
print(response)

content='1. GPT-3 (Generative Pre-trained Transformer 3) - Entwickelt von OpenAI, hat 175 Milliarden Parameter und gilt momentan als das größte Sprachmodell.\n\n2. GPT-2 (Generative Pre-trained Transformer 2) - Auch von OpenAI entwickelt, hat 1,5 Milliarden Parameter und war vor GPT-3 das größte verfügbare Sprachmodell.\n\n3. T5 (Text-to-Text Transfer Transformer) - Entwickelt von Google Research, hat 11 Milliarden Parameter und wurde für eine Vielzahl von Aufgaben wie Übersetzung, Textgenerierung und Frage-Antwort verwendet.\n\n4. Turing NLG - Entwickelt von Microsoft Research, hat 17 Milliarden Parameter und wurde speziell für die natürliche Sprachgenerierung entwickelt.\n\n5. Megatron-LM - Entwickelt von NVIDIA, hat 8,3 Milliarden Parameter und wurde für verschiedene Aufgaben im Bereich des maschinellen Lernens eingesetzt, darunter auch die Sprachverarbeitung.'
content='Von den genannten Sprachmodellen stammen GPT-3 und GPT-2 von OpenAI. OpenAI ist ein Unternehmen für künstliche Int

Erstmal sehen wir, dass wir unterschiedliche Message-Typen haben: SystemMessage, Humanmessage und AIMessage. Diese widerspiegeln die verschiedenen Rollen, die von ChatGPT erwartet werden.
Der Ablauf ist stets so, dass wir der Liste der Nachrichten die Fragen und Antworten hinzufügen. Die _messages_ Liste ist somit unser Chatverlauf und praktisch unser Speicher.

Damit wir das nicht manuell machen müssen, bietet LangChain _Memory_ an. Davon gibt es unterschiedliche Implementierungen. Der einfachste Speicher merkt sich ganz einfach jede Nachricht, so wie wir es oben gemacht haben. Allerdings ist der Platz im Kontext begrenzt und ein Chat, der länger geführt wird, könnte dazu führen, dass die maximale Kontextgröße überschritten wird. Dafür gibt es Speicher, die entweder nur die letzten n Nachrichten behält oder noch ausgefeilter, ein Speicher, der die bisherige Kommunikation zusammenfasst, um Platz zu sparen.

Schauen wir uns das Beispiel von oben mit LangChain Memory an:

In [2]:
from langchain.memory import ChatMessageHistory

llm = ChatOpenAI()

history = ChatMessageHistory()

history.add_message(SystemMessage(content="Du bist ein hilfreicher KI Assistent, der mir bei meinen Fragen und Aufgaben hilft"))

history.add_user_message("Erstelle bitte eine Liste der fünf größten Sprachmodelle")

response = llm(history.messages)

history.add_ai_message(response.content)

print(response)

content='Hier ist eine Liste der fünf größten Sprachmodelle:\n\n1. GPT-3 (Generative Pretrained Transformer 3) - Entwickelt von OpenAI, besteht GPT-3 aus 175 Milliarden Parametern und gilt derzeit als eines der größten Sprachmodelle der Welt.\n\n2. Turing NLG (Natural Language Generation) - Entwickelt von Microsoft, basiert auf GPT-3 und hat ebenfalls eine beeindruckende Größe von 17 Milliarden Parametern.\n\n3. T5 (Text-to-Text Transfer Transformer) - Entwickelt von Google, besteht T5 aus 11 Milliarden Parametern und zeichnet sich durch seine Fähigkeit aus, Texte in verschiedene Sprachen zu übersetzen.\n\n4. Megatron-LM - Entwickelt von NVIDIA, ist Megatron-LM ein Sprachmodell mit 8,3 Milliarden Parametern und wurde speziell für die Verarbeitung von natürlicher Sprache entwickelt.\n\n5. ELECTRA (Efficiently Learning an Encoder that Classifies Token Replacements Accurately) - Entwickelt von Google, hat ELECTRA rund 134 Millionen Parameter und zeichnet sich durch seine effiziente Traini

Das Beispiel soll zeigen, wie die Grundstrukturen eines Speichers aussehen. Im Grunde genommen ist das mit den manuellen Schritten erstmal fast identisch und hilft uns erstmal nicht das Ganze zu automatisieren.
In Kombination mit einer _vernünftigen_ Speicher-Implementierung und einer Chain sieht das Ganze wesentlich besser aus:

In [3]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()

llm = ChatOpenAI()

conversation = ConversationChain(
    llm=llm, 
    memory=memory
)

response = conversation.predict(input="Welche Bedeutung hat die Angabe der Anzahl der Parameter bei einem Sprachmodell?")

print(response)

Die Angabe der Anzahl der Parameter bei einem Sprachmodell gibt Auskunft über die Komplexität und Größe des Modells. Ein Parameter ist eine Variable, die das Modell verwendet, um Informationen zu speichern oder zu verarbeiten. Je mehr Parameter ein Modell hat, desto mehr Informationen kann es verarbeiten und desto komplexere Zusammenhänge kann es erfassen. Die Anzahl der Parameter kann auch einen Einfluss auf die Leistung des Modells haben. Ein Modell mit mehr Parametern kann in der Regel mehr Daten und Informationen berücksichtigen, was zu einer höheren Genauigkeit führen kann. Allerdings kann ein Modell mit zu vielen Parametern auch zu Overfitting führen, bei dem das Modell die Trainingsdaten zu stark "auswendig lernt" und bei neuen Daten schlechter abschneidet. Es ist also wichtig, ein ausgewogenes Verhältnis zwischen der Anzahl der Parameter und der Leistung des Modells zu finden.


In [4]:
response = conversation.predict(input="Wie viele sind es bei GPT-3?")

print(response)

GPT-3 hat insgesamt 175 Milliarden Parameter. Das ist eine enorme Anzahl und macht GPT-3 zu einem der größten und komplexesten Sprachmodelle, die bisher entwickelt wurden. Mit dieser großen Anzahl an Parametern kann GPT-3 eine breite Palette an Informationen und Zusammenhängen erfassen und komplexe Aufgaben im Bereich der natürlichen Sprachverarbeitung bewältigen.


Wir sehen also, dass die Chain sich automatisch darum kümmert die Nachrichten in den dazugehörigen Speicher zu laden. Durch die Verwendung von _ConversationChain_ haben wir automatisch ein Prompttemplate, das einen _input_ Parameter erwartet, den wir an die _predict_ Funktion übergeben.

In [5]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Welche Bedeutung hat die Angabe der Anzahl der Parameter bei einem Sprachmodell?'), AIMessage(content='Die Angabe der Anzahl der Parameter bei einem Sprachmodell gibt Auskunft über die Komplexität und Größe des Modells. Ein Parameter ist eine Variable, die das Modell verwendet, um Informationen zu speichern oder zu verarbeiten. Je mehr Parameter ein Modell hat, desto mehr Informationen kann es verarbeiten und desto komplexere Zusammenhänge kann es erfassen. Die Anzahl der Parameter kann auch einen Einfluss auf die Leistung des Modells haben. Ein Modell mit mehr Parametern kann in der Regel mehr Daten und Informationen berücksichtigen, was zu einer höheren Genauigkeit führen kann. Allerdings kann ein Modell mit zu vielen Parametern auch zu Overfitting führen, bei dem das Modell die Trainingsdaten zu stark "auswendig lernt" und bei neuen Daten schlechter abschneidet. Es ist also wichtig, ein ausgewoge

Die Ausgabe der _memory_ Variable zeigt, wie die ganze Konversation mitgeschnitten wurde.

Jetzt sind wir praktisch nur eine Schleife weg von einem Chatbot:

In [6]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI

memory = ConversationBufferMemory()

llm = ChatOpenAI()

conversation = ConversationChain(
    llm=llm, 
    memory=memory
)

while True:
    prompt = input("Prompt:")
    response = conversation.predict(input=prompt)
    print(response)

Mir geht es gut, danke! Als KI habe ich keine Gefühle im eigentlichen Sinne, aber ich bin bereit, mit dir zu chatten und zu helfen. Wie kann ich dir behilflich sein?
Natürlich! Ich kann dir gerne alles über KIs erklären. Künstliche Intelligenz ist ein Bereich der Informatik, der sich mit der Entwicklung von Systemen befasst, die in der Lage sind, Aufgaben auszuführen, die normalerweise menschliche Intelligenz erfordern würden. KIs verwenden Algorithmen und Daten, um Muster zu erkennen, Entscheidungen zu treffen und Probleme zu lösen.

Es gibt verschiedene Arten von KIs, wie zum Beispiel schwache KIs und starke KIs. Schwache KIs sind auf spezifische Aufgaben oder Probleme spezialisiert, wie zum Beispiel Gesichtserkennung oder Sprachverarbeitung. Starke KIs hingegen sind in der Lage, eine Vielzahl von Aufgaben zu erledigen und sogar menschenähnliche Intelligenz zu zeigen.

Künstliche Intelligenz hat viele Anwendungen in verschiedenen Bereichen wie Medizin, Finanzen, Verkehr, Robotik und 

KeyboardInterrupt: Interrupted by user

Einmal hübsch machen, Weboberfläche, fertig ist der Chatbot!

Jetzt kann es sein, dass wenn wir viel Eingeben (und ausgeben lassen), das irgendwann der Kontext zu voll wird. Wie oben erwähnt können wir dafür verschiedene Speicherarten nutzen.

Dieser hier begrenzt den Chat auf ein bestimmtes Token-Limit (Token: Sprachmodelle arbeiten mit Tokens anstelle von Wörtern. Manche Wörter, insbesondere in Deutsch, bestehen aus mehreren Tokens.)

In [7]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()


# Wir übergeben das Sprachmodell + Maximale Anzahl an Tokens. Das Sprachmodell dient dazu die Anzahl der Tokens zu ermitteln
# Hier wir zu Demonstrationszwecken sehr kleines Limit angegeben
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=100)

conversation = ConversationChain(
    llm=llm, 
    memory=memory
)

response = conversation.predict(input="Hallo!")
print(f"Antwort: {response}")
print(f"Der Inhalt des Speichers jetzt:\n{memory.buffer_as_str}\n\n")
    
response = conversation.predict(input="Erstelle mir bitte eine Liste der letzten 10 Präsidenten der USA")
print(f"Antwort: {response}")
print(f"Der Inhalt des Speichers jetzt:\n{memory.buffer_as_str}\n\n")

response = conversation.predict(input="Welcher davon ist der aktuelle Präsident?")
print(f"Antwort: {response}")
print(f"Der Inhalt des Speichers jetzt:\n{memory.buffer_as_str}\n\n")

Antwort: Hello! How can I assist you today?
Der Inhalt des Speichers jetzt:
Human: Hallo!
AI: Hello! How can I assist you today?


Antwort: Natürlich! Hier sind die letzten 10 Präsidenten der USA:

1. Joe Biden (2021-heute)
2. Donald Trump (2017-2021)
3. Barack Obama (2009-2017)
4. George W. Bush (2001-2009)
5. Bill Clinton (1993-2001)
6. George H. W. Bush (1989-1993)
7. Ronald Reagan (1981-1989)
8. Jimmy Carter (1977-1981)
9. Gerald Ford (1974-1977)
10. Richard Nixon (1969-1974)

Gibt es noch etwas, womit ich Ihnen helfen kann?
Der Inhalt des Speichers jetzt:



Antwort: Der aktuelle Präsident ist Joe Biden. Er wurde am 20. Januar 2021 als 46. Präsident der Vereinigten Staaten vereidigt. Er ist Mitglied der Demokratischen Partei und diente zuvor als Vizepräsident unter Barack Obama. Biden hat eine lange politische Karriere hinter sich und war auch acht Jahre lang Senator für Delaware.
Der Inhalt des Speichers jetzt:
AI: Der aktuelle Präsident ist Joe Biden. Er wurde am 20. Januar 2021

Zu guter letzt ein schauen wir uns einen etwas ausgefeilteren Speicher an. Dieser löscht nicht einfach alte Konversationen, sondern fasst den bisherigen Verlauf zusammen.
Zu beachten ist, dass das zusätzliche Aufrufe zu unserem Sprachmodell bedeuten und somit auch weitere Kosten verursachen kann!

In [8]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()


# Wir übergeben das Sprachmodell. Das Sprachmodell dient dazu die Zusammenfassung zu erstellen.
memory = ConversationSummaryMemory(llm=llm)

conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

response = conversation.predict(input="Hallo!")
print(f"Antwort: {response}")
    
response = conversation.predict(input="Erstelle mir bitte eine Liste der letzten 10 Präsidenten der USA")
print(f"Antwort: {response}")

response = conversation.predict(input="Welcher davon ist der aktuelle Präsident?")
print(f"Antwort: {response}")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hallo!
AI:

> Finished chain.
Antwort: Hallo! Wie kann ich Ihnen helfen?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human greets the AI in German. The AI responds in German and asks how it can help.
Human: Erstelle mir bitte eine Liste der letzten 10 Präsidenten der USA
AI:

> Finished chain.
Antwort: Hallo! Natürlich kann ich das für dich tun. Hier ist eine Liste der letzten 10 Prä